# Ensemble learning

Ensemble learning is een manier om meerdere machine learning algorithmes te gaan combineren om een beter resultaat te bekomen.
Een voorbeeld hiervan is Random Forest dat een ensemble is van een aantal decision trees. 
Een belangrijke opmerking hierbij is dat meerdere soorten machine learning algoritmes gecombineerd kunnen worden.
Er zijn hier drie bekende varianten van, namelijk:
* Stacking
* Bagging
* Boosting

Deze varianten gaan we nu 1 voor 1 behandelen.

# Stacking 

Een schets van hoe een stacking-ensemble werkt zie je hieronder.
Bij de stacking-methode wordt de volledige trainingsdata gebruikt om meerdere modellen te trainen.
Deze kunnen maar moeten niet van hetzelfde algoritme zijn en ook de hyperparameters kunnen verschillen.
Nadat al deze modellen getrained zijn, wordt er nog model getrained dat een selectie maakt van wat de uiteindelijke voorspelling is op basis van de uitkomst van alle getrainde modellen.
Dit tweede model kan dus evalueren wanneer welk model het meest correct is.
Het algoritme hiervoor kan vrij gekozen worden en kan zelfs een ander ensemble zijn.
Bij stacking is dus niet strikt majority voting van toepassing.

![ensemble](images\stacking.png)

Een [stacking ensemble](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) kan als volgt geimplementeerd worden in sklearn:

In [4]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# stack met estimators
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('lr', LogisticRegression(C=100)),
    ('svc', SVC(C=10)),
    ('knn', KNeighborsClassifier(n_neighbors=3))
]

# maak de stacking classifier met de bovenstaande stack
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)  # stratify zorgt ervoor dat het percentage van klasses van X en y behouden blijft in de X_train, y_train
stack.fit(X_train, y_train)
stack.score(X_test, y_test)


0.9473684210526315

# Bagging

De tweede methode bagging is ook gekend onder de naam bootstrap bagging.
Dit houdt in dat elk model slechts getrained wordt op een deel van de data (bootstrapped samples).
Deze observaties om elk model te trainen worden willekeurig gekozen met teruglegging.
Typisch wordt er ongeveer 60% van de totale data gebruikt om elk model te trainen.
Wanneer elk model getrained is wordt er een standaard majority voting toegepast om de uiteindelijke predictie te bekomen (bij regressie wordt er het gemiddelde genomen).

![bagging](images\bagging.png)

Een belangrijke opmerking is dat theoretisch gezien meerdere types van machine learning algoritmes gecombineerd kunnen worden maar dit dit niet rechtstreeks gaat met sklearn zonder zelf een ensemble te implementeren. 
Dit is ook niet nodig omdat zolang je ensemble groot genoeg is, je elke gewenste accuraatheid kan bereiken.
Een implementatie van de [bagging methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) ziet er als volgt uit:

In [7]:
# create bagging classifier
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=10, random_state=0)

%time clf.fit(X_train, y_train)     # %time is een jupyter notebook tool om de tijd bij te houden hoe lang een commando duurt
%time clf.score(X_test, y_test)

CPU times: total: 15.6 ms
Wall time: 7 ms
CPU times: total: 0 ns
Wall time: 4.01 ms


0.9473684210526315

# Boosting

De derde variant van ensemble learning methoden is boosting.
Dit is een aanpassing van de bagging methode waar de bootstrapped samples niet meer willekeurig zijn.
Bij boosting worden de modellen sequentieel getrained en gevalideerd (met de trainingsdata diet niet in de bootstrapped sample zit). 
De classificaties die verkeerd waren bij deze validatie stap hebben een grotere kans om in de bootstrapped sample te zitten van het volgende model.
Omdat de uitkomst van elk model nodig is voor het volgende kan dit niet geparallelliseerd worden waardoor de trainingstijd snel kan oplopen.
Het voordeel echter van deze methode is dat de accuraatheid van het gecombineerde model hoger gaat zijn dan bij bagging.
De meest bekende implementatie van deze methode wordt AdaBoost genoemd.
Daarnaast wordt tegenwoordig ook XGBoost benoemd wat staat voor Extreme Gradient Boosting.
Het probleem is echter dat deze techniek niet standaard in sklearn staat.
Hiervoor moet een extra package toegevoegd worden, meer informatie hierover vind je [hier](https://towardsdatascience.com/getting-started-with-xgboost-in-scikit-learn-f69f5f470a97).
Deze techniek is een speciale variant van een random forest (er wordt gewerkt met meerdere decision trees).
Meer informatie over de api vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier).
De voordelen van xgboost zijn:
* Hoge accuraatheid
* Heel snelle uitvoering door parallellisatie
* Flexibel algoritme door keuse van optimalisatie
* Kan omgaan met missing data
* Voert autmatisch pruning uit om overfitting tegen te gaan
* Ingebouwde cross-validatie

![boosting](images\boosting.png)

 Een implementatie van de [boosting methode](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) ziet er als volgt uit:

In [14]:
# create boosting classifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=10, random_state=0)

%time clf.fit(X_train, y_train)     # %time is een jupyter notebook tool om de tijd bij te houden hoe lang een commando duurt
%time clf.score(X_test, y_test)

CPU times: total: 46.9 ms
Wall time: 24 ms
CPU times: total: 0 ns
Wall time: 2 ms


0.9210526315789473

Meer informatie over alle mogelijke ensemble-methoden in sklearn vind je [hier](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble).

# XGBoost

Gradient Boosting is een ML-techniek die iteratief modellen toevoegt aan een ensemble.
Er wordt begonnen met een eenvoudig model dat zeer naief is.
De voorspellingen van dit model worden gebruikt om een kost-functie te berekenen.
Deze functie wordt dan gebruikt voor een nieuw model te trainen dat toegevoegd aan het ensemble.
De parameters van dit nieuwe model worden zo gekozen dat de kost-functie verminderd.
Met dit uitgebreide ensemble worden opnieuw voorspelling gedaan die het dan terug gebruikt worden om een kostfunctie te berekenen.

De belangrijkste parameters van deze techniek zijn:
* n_estimators: Geeft het maximum aantal bomen in het ensemble weer. Een lage waarde kan leiden tot underfitting, een grote tot overfitting.
* learning rate: Dit kan gebruikt zorden om de impact van extra bomen te verkleinen om overfitting tegen te gaan. Lagere waarden gaan normaal een hogere accuraatheid maar ook een hogere trainingstijd opleveren.
* early stopping: Deze parameter moet meegegeven worden bij het fitten en kan gebruikt worden om overfitting tegen te gaan. Indien de validation score niet afneemt gedurende dit aantal rondes, dan stopt de fit-functie omdat er overfitting gedetecteerd wordt.
* eval_set: Deze parameter is een tuple van de validatieset dat gebruikt kan worden voor het early-stopping te testen. Deze moet dus met early-stopping gecombineerd worden.
* n_jobs: aantal cores dat kan gebruikt worden voor de training


Meer informatie over de parameters van deze techniek vind je [hier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn).

In [16]:
# create xgboost classifier
from xgboost import XGBClassifier

clf = XGBClassifier(n_estimators=1000, learning_rate=0.05)      
# learning rate: de grootte van de stappen om tot het optimum te komen: te laag - duurt lang om te trainen, te hoog en de training kan onstabiel worden
clf.fit(X_train, y_train, early_stopping_rounds= 5, eval_set=[(X_test, y_test)])

clf.score(X_test, y_test)

[0]	validation_0-mlogloss:1.03444
[1]	validation_0-mlogloss:0.97572
[2]	validation_0-mlogloss:0.92178
[3]	validation_0-mlogloss:0.87210
[4]	validation_0-mlogloss:0.82620
[5]	validation_0-mlogloss:0.78371
[6]	validation_0-mlogloss:0.74428
[7]	validation_0-mlogloss:0.70764
[8]	validation_0-mlogloss:0.67353
[9]	validation_0-mlogloss:0.64173
[10]	validation_0-mlogloss:0.61205
[11]	validation_0-mlogloss:0.58431
[12]	validation_0-mlogloss:0.55837
[13]	validation_0-mlogloss:0.53407
[14]	validation_0-mlogloss:0.51131
[15]	validation_0-mlogloss:0.48996
[16]	validation_0-mlogloss:0.46993
[17]	validation_0-mlogloss:0.45111
[18]	validation_0-mlogloss:0.43344
[19]	validation_0-mlogloss:0.41683
[20]	validation_0-mlogloss:0.40121
[21]	validation_0-mlogloss:0.38652
[22]	validation_0-mlogloss:0.37269
[23]	validation_0-mlogloss:0.35967
[24]	validation_0-mlogloss:0.34741
[25]	validation_0-mlogloss:0.33587
[26]	validation_0-mlogloss:0.32500
[27]	validation_0-mlogloss:0.31475
[28]	validation_0-mlogloss:0.3

c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.9473684210526315